# Regional Analysis

In this notebook we will explore the results of the three previous notebooks in terms of specific regions in the US. We will look at the High Plains Aquifer, which has been shown to have accelerated decline in aquifer storage, the Central Valley, as it is another region that has suffered from declines in aquifer storage due to agricultural usage, and the Upper Colorado River Basin, as it is a critical basin for understanding the down stream flow of the Colorado River in the southwest US.

In [ ]:
import geopandas as gpd
import pandas as pd
from shapely import unary_union, box
from shapely.geometry import Polygon
from scipy.stats import norm, kstest
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import panel as pn
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
from xarray_einstats import linalg, stats
import sciencebasepy
import os
import zipfile
import itertools
import warnings

## Data Preparation

To begin, let's read in the data we will need:

  1. the region shapefiles,
  2. the regridded ET data for each data set,
  3. the TC error variances and SNR estimates,
  4. the EC error covariance and cross-correlation estimates, and
  5. the relative bias of each data set to the other data sets.

### Region Shapefiles

We will start with the region shapefiles ([High Plains aquifer](https://www.sciencebase.gov/catalog/item/6314061bd34e36012efa397b), [Central Valley](https://www.sciencebase.gov/catalog/item/63140570d34e36012efa2c01), and [Upper Colorado River Basin](https://www.sciencebase.gov/catalog/item/4f4e4a38e4b07f02db61cebb)), which we will download from USGS ScienceBase.

In [ ]:
if not os.path.isdir('../Data/regions'):
    os.mkdir('../Data/regions')

sciencebase_regions = {'High Plains aquifer': '6314061bd34e36012efa397b',
                       'Central Valley': '63140570d34e36012efa2c01',
                       'Upper Colorado River Basin': '4f4e4a38e4b07f02db61cebb'}

for region in sciencebase_regions:
    filename = region.replace(' ', '_')

    # Only download file if we don't already have it
    if not os.path.isfile(f'../Data/regions/{filename}.zip'):
        # Establish a ScienceBase session.
        sb = sciencebasepy.SbSession()
    
        # Get list of files on the ScienceBase page
        file_list = sb.get_item_file_info(sb.get_item(sciencebase_regions[region]))

        # If the ScienceBase page has the shapefile in zip format, get that.
        # Otherwise, download everything and place it into a zip file
        zip_url = [f['url'] for f in file_list if 'zip' in f['name']]
        if zip_url:
            sb.download_file(zip_url[0], f'{filename}.zip', '../Data/regions/')
        else:
            response = sb.get_item_files(sb.get_item(sciencebase_regions[region]),
                                             '../Data/regions/')
            # zipfile requires you to work in the directory containing the files...
            cwd = os.getcwd()
            os.chdir('../Data/regions/')
            with zipfile.ZipFile(f'{filename}.zip', 'w') as myzip:
                for file in [item['name'] for item in response]:
                    myzip.write(file)
                    os.remove(file)

            os.chdir(cwd)

Now that we have the region files, we will read each of them in, convert them to a coordinate reference system of equal distance, and combine them into a single data frame.

In [ ]:
high_plns_aqfr = gpd.read_file('../Data/regions/High_Plains_aquifer.zip')

# Exclude the non-aquifer regions
high_plns_aqfr = high_plns_aqfr[high_plns_aqfr['AQUIFER'] == 'High Plains aquifer']
# Combine the polygons into one single MultiPolygon
high_plns_aqfr = gpd.GeoDataFrame(geometry=[unary_union(high_plns_aqfr.geometry.values)],
                                  crs=high_plns_aqfr.crs)
high_plns_aqfr = high_plns_aqfr.to_crs('EPSG:4269').to_crs('EPSG:4326')
high_plns_aqfr['region_name'] = 'High Plains Aquifer'

cntrl_valley = gpd.read_file('../Data/regions/Central_Valley.zip')
# Buffer is needed to ensure boundary lines of Major Areas overlap.
# CRS is in units of meters, so we only buffer by 1mm
cntrl_valley = gpd.GeoDataFrame(geometry=[unary_union(cntrl_valley.geometry.buffer(0.001).values)],
                                crs=cntrl_valley.crs)
cntrl_valley = cntrl_valley.to_crs('EPSG:4269').to_crs('EPSG:4326')
cntrl_valley['region_name'] = 'Central Valley'

ucrb = gpd.read_file('../Data/regions/Upper_Colorado_River_Basin.zip')

# Drop the unneeded variables
ucrb = ucrb.drop(columns=['EXT_ID', 'EXT_TYP_ID', 'NAME'])
# UCRB CRS is in 4326 already, but let's add this in just in case
ucrb = ucrb.to_crs('EPSG:4326')
ucrb['region_name'] = 'Upper Colorado River Basin'

# Group the regions into one DataFrame
regions = pd.concat([high_plns_aqfr, cntrl_valley, ucrb], ignore_index=True)
regions

### Regridded ET data

Next, we will read in the regridded ET data and combine the data sets into one `xarray.Dataset`. We will not reduce the data sets to a common data range like the previous notebooks. Instead, we will keep all the data so we can get a time series of the total ET in each region.

In [ ]:
files = ['../Data/ssebop/ssebop_aet_regridded.nc',
         '../Data/gleam/gleam_aet.nc',
         '../Data/era5/era5_aet_regridded.nc',
         '../Data/nldas/nldas_aet_regridded.nc',
         '../Data/terraclimate/terraclimate_aet_regridded.nc',        
         '../Data/wbet/wbet_aet_regridded.nc',
         ]
dataset_name = ['SSEBop', 'GLEAM', 'ERA5', 'NLDAS', 'TerraClimate', 'WBET']

ds_et = xr.open_mfdataset(files, engine='netcdf4', combine='nested', concat_dim='dataset_name')
ds_et = ds_et.assign_coords({'dataset_name': dataset_name})
ds_et.dataset_name.attrs['description'] = 'Dataset name'
ds_et.aet.attrs['description'] = 'Actual evapotranspiration, monthly total'
ds_et.aet.attrs['standard_name'] = 'Actual evapotranspiration'
ds_et.aet.attrs['long_name'] = 'Actual evapotranspiration'

# Extract the date range of each data set for later
date_ranges = {}
for file, name in zip(files, dataset_name):
    ds_temp = xr.open_dataset(file, engine='netcdf4', chunks={'lon': -1, 'lat': -1, 'time': -1})
    date_ranges[name] = [ds_temp.time.min().values, ds_temp.time.max().values]

# Transpose to have time first
ds_et = ds_et.transpose('time', ...)
# The data set is less than 2GiB, so let's read it into memory vs keeping as a dask array
ds_et = ds_et.compute()
ds_et

### Collocation and Bias Results

Since we formatted the TC, EC, and Bias data before saving, these are quick and easy reads. We can simply read them in and combine them into one results `Dataset` after simply renaming some variables and coordinates.

In [ ]:
ds_tc = xr.open_dataset('../Data/TC_errs.nc', engine='netcdf4')
ds_ec = xr.open_dataset('../Data/EC_errs.nc', engine='netcdf4')
ds_bias = xr.open_dataset('../Data/avg_bias.nc', engine='netcdf4')
ds_results = xr.merge([ds_tc.rename({'est_idx': 'var_est_idx', 'est_pair': 'var_est_pair'}),
                       ds_ec.rename({'covar_pair': 'dataset_pair'}),
                       ds_bias])
ds_results.attrs = None
ds_results.dataset_pair.attrs['description'] = 'Data set pair used in the EC or relative bias evaluation.'
ds_results

Now that we have read in our data, let's make a plot that shows the regions overlaid on the ET data. That way we have an idea of the location of the regions.

In [ ]:
et_data_snapshot = ds_et.aet.sel(dataset_name='SSEBop', time='2001-07').squeeze()
plt = (et_data_snapshot.hvplot(x='lon', y='lat', geo=True, coastline=True)
       * regions.hvplot(c='region_name', alpha=0.5))
plt.opts(frame_width=500, frame_height=250)

## Creating a Weight Map

To do a regional analysis, we need to be able to aggregate our gridded data products over each region. This aggregation requires a weight map that shows the fractional area of each grid pixel that is within the regions. This is the same idea as the conservative regridding that we used to regrid the ET data to a common grid back in the [regridding notebook](1_regrid.ipynb). The only difference is rather than going from a rectangular region to a rectangular region (i.e. pixel to pixel), we are going from a rectangular region to a complex polygon. Unfortunately, we cannot currently use `xarray.regrid` for this, as it only works with rectilinear regridding. However, it looks like it may be a possibility in the future ([Issue #36](https://github.com/EXCITED-CO2/xarray-regrid/issues/36)). In the mean time, we will create a process that allows for regional aggregation and the creation of a weight map following the general methods discussed in [this Pangeo Discourse](https://discourse.pangeo.io/t/conservative-region-aggregation-with-xarray-geopandas-and-sparse/2715).

First, let's start be defining a function that converts our grid to polygon boxes.

In [ ]:
def grid_to_poly_boxes(grid, lat_coord='latitude', lon_coord='longitude'):
    """
    Creates a collection of polygons that define each grid cell.
    
    Parameters
    ----------
    grid : Dataset or DataArray
        The grid that has the grid points defined in its coordinates.
    lat_coord: str
        The coordinate in ``grid`` that defines the latitude at the center
        of each grid cell.
    lon_coord : str
        The coordinate in ``grid`` that defines the longitude at the center
        of each grid cell.
    
    Returns
    -------
    boxes : DataArray
        A collection of polygon boxes that define each grid cell.
    """
    # Get the bounds of the grid from the center points.
    # NOTE: this assumes the grid is rectilinear!!!
    lat_diff = grid[lat_coord].diff(dim=lat_coord)
    if len(np.unique(lat_diff)) != 1:
        raise ValueError('Latitude grid is not equally spaced.')
    else:   
        lat_spacing = np.abs(np.unique(lat_diff))
        
    lon_diff = grid[lon_coord].diff(dim=lon_coord)
    if len(np.unique(lon_diff)) != 1:
        raise ValueError('Longitude grid is not equally spaced.')
    else:   
        lon_spacing = np.abs(np.unique(lon_diff))
        
    # Generate the bounds on the grid
    bounds = np.vstack((grid[lat_coord] + lat_spacing/2,
                        grid[lat_coord] - lat_spacing/2)).T
    grid[lat_coord+'_bounds'] = ((lat_coord, 'bound'), bounds)
    bounds = np.vstack((grid[lon_coord] + lon_spacing/2,
                        grid[lon_coord] - lon_spacing/2)).T
    grid[lon_coord+'_bounds'] = ((lon_coord, 'bound'), bounds)

    # Convert the grid and its bounds to a collection of points
    points = grid.stack(point=(lat_coord, lon_coord))
    
    # Create polygons from the grid bounds
    def bounds_to_poly(lat_bounds, lon_bounds):
        if lon_bounds[0] >= 180:
            # geopandas needs this as it goes from -180 to 180
            lon_bounds = lon_bounds - 360
        return box(lon_bounds[0],
                   lat_bounds[0],
                   lon_bounds[1],
                   lat_bounds[1])

    boxes = xr.apply_ufunc(
        bounds_to_poly,
        points[lat_coord+'_bounds'],
        points[lon_coord+'_bounds'],
        input_core_dims=[('bound',),  ('bound',)],
        output_dtypes=[np.dtype('O')],
        vectorize=True
    )

    return boxes

Now that we have the function to create the polygons, we can convert the grid (which is the same for all data sets since we regridded to a common grid) to the polygon boxes.

In [ ]:
# Select the lat/lon grid from the ET data set
grid = ds_et[['lat', 'lon']]
boxes = grid_to_poly_boxes(grid, lat_coord='lat', lon_coord='lon')

# Place the boxes into a geodataframe
grid_df= gpd.GeoDataFrame(
    data={'geometry': boxes.data, 'lat': boxes.lat, 'lon': boxes.lon},
    index=boxes.indexes['point'],
    crs='EPSG:4326'
)
grid_df

With the polygon boxes, we are now ready to make our weight map. We can do this simply with [`geopandas.overlay`](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.overlay.html#geopandas.GeoDataFrame.overlay), which overlays our polygon boxes over the region polygons, thereby only keeping the boxes and fraction of boxes that are contained with in the polygons. With the overlay, we can then extract the area of each box and divide it by the total area of the region to get the box's (i.e. pixel's) area weight.

In [ ]:
# Use an area preserving projection
equal_area_crs = 'EPSG:5070'

overlay = grid_df.to_crs(equal_area_crs).overlay(regions.to_crs(equal_area_crs))

# Get the grid box fractional area
grid_cell_fraction = (
    overlay.geometry.area.groupby(overlay['region_name'])
    .transform(lambda x: x / x.sum())
)

# Place the fractional areas back into a xarray.Dataset with the same
# lat/lon grid as the original data.
multi_index = overlay.set_index(['lat', 'lon', 'region_name']).index
df_weights = pd.DataFrame({'weights': grid_cell_fraction.values}, index=multi_index)
# Place in Dataset and extract the MultiIndex to dimensions
# We could convert to a sparse matrix by adding `sparse=True` to the unstack
# call, but there is currently no need as our grid is small (~700 kiB).
da_weights = xr.Dataset(df_weights).unstack(fill_value=0).weights
# Align the weight grid to the data grid, as the weight grid
# currently only has the weights in the regions
da_weights, _ = xr.align(da_weights, ds_et, join='outer',
                         exclude=['time', 'dataset_name'],
                         fill_value=0)
da_weights

## Regional Aggregation

### Regridded ET data

Now that we have our weighted grid for each region, we can aggregate our data however we need. As noted in a comment line above, we could make the weights a sparse matrix, but we don't since the weight grid is small. One other advantage of not using a sparse matrix is that we can take advantage of [`xarray.Dataset.weighted()`](https://docs.xarray.dev/en/stable/user-guide/computation.html#weighted-array-reductions). This simplifies the aggregation as we do not need to use the dot product as in the [Pangeo Discourse](https://discourse.pangeo.io/t/conservative-region-aggregation-with-xarray-geopandas-and-sparse/2715).

First, let's start by aggregating the regridded ET data. Since our weights for each region were designed to sum to 1, we can easily compute the area weighted average ET of the regions using `Dataset.weighted()`.

In [ ]:
weighted_et = ds_et.weighted(da_weights)
ds_et_regional = weighted_et.mean(dim=['lat', 'lon'], skipna=True, keep_attrs=True)

ds_et_regional.time.attrs = ds_et.time.attrs
ds_et_regional.dataset_name.attrs = ds_et.dataset_name.attrs
ds_et_regional.region_name.attrs['description'] = 'Region name'
ds_et_regional.aet.attrs['description'] = 'Regionally aggregated total actual evapotranspiration'
ds_et_regional

With the computed regional ET estimates, let's go ahead and plot the time series and see how they look. 

In [ ]:
plt = ds_et_regional.aet.hvplot(
    groupby=['dataset_name', 'region_name'], label='Total'
).overlay('dataset_name').opts(legend_position='right', frame_width=500)

pn.panel(plt, widget_location='top')

From the time series for the High Plains and Upper Colorado, we can see that each ET data set is relatively consistent. All six data sets have overlapping ranges in ET values, indicating that none are estimating ET values that are highly inconsistent with the other data sets. However, when looking at the Central Valley, we can see that the WBET data set has a noticable decrease in average ET in the Central Valley around 1980. After reviewing the paper describing the WBET data set [(Reitz et al. 2023)](https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2022WR034012) and remembering that the Central Valley is a highly irrigated region, this decrease is due to how the WBET data set implements irrigation into its water balance calculations. The calculation only includes irrigation estimates from 1980 to 2018. As per the paper "Because  we  lacked  consistent  national-scale  data  sets for years prior to 1980, those years do not include estimates for irrigation". Since the Central Valley is so heavily irrigated, this inclusion of irrigation is most pronounced for this region compared to the High Plains Aquifer and Upper Colorado River Basin. 

Another interesting difference in the data sets for the Central Valley can be seen when zooming in on the 2000s. Here, we can see that the peak ET month is shifted earlier in the year (~April) for GLEAM, ERA5, NLDAS, and TerraClimate, whereas SSEBop and WBET have the peak ET around July, which is the peak month for the High Plains and Upper Colorado for all data sets. The reason for this likely stems from the precipitation patterns in the Central Valley and the lack of an irrigation component in these four ET data sets. In the Central Valley, precipitation primarily occurs in the winter months and is minimal in the summer. To account for this lack of precipitation, the region is highly irrigated. Therefore, based on weather alone, ET should peak in the spring as temperatures start to rise and the precipitation begins to decrease. However, if irrigation is included, the amount of ET should continue to rise as the temperature does, peaking in the summer months since water is continually supplied through irrigation. So, including an irrigation component when calculating ET can affect monthly timings of peak ET. Irrigation being the likely cause is further supported by the peak shift only occuring in the WBET data after 1980, when irrigation is included.

Besides this temporal offset in the Central Valley, there is one other interesting thing to note. It is that GLEAM and NLDAS typically have more limited ranges in ET compared to the others in all regions. This comparatively limited range could be one reason that these two data sets have the lowest TC estimated error variances of the six data sets (i.e., lower range would mean lower variance, which would result in lower TC errors). However, the affine error model assumed by our TC implementation should account for any scaling differences, if the data sets are truly independent. Since we found in [the EC notebook](3_EC_application.ipynb#EC-Discussion) that the data sets may not be fully independent, it could be that these lower ranges are resulting in artifically lower error variances. Therefore, it is safest to conclude that error variances estimated by TC are lower limits on the true errors.

### Collocation and Bias Results

Next, we can apply the same aggregation method as we did with the regridded ET data set on the collocation and bias results. This is just as easy as the regirdded data since we use ``xarray.weighted``. One thing we must note and update though, is the aggregation of the TC error variances (i.e., ``error`` in ``ds_results``). Since we saved these as error standard deviations, we must square them back to variances before summing.

In [ ]:
weighted_results = ds_results.drop_vars(['error']).weighted(da_weights)
weighted_errors = (ds_results[['error']] ** 2).weighted(da_weights)

# Use mean rather than sum as it accounts for NaNs
# The are equivalent since the sum of weights is 1
ds_et_results = weighted_results.mean(dim=['lat', 'lon'], skipna=True, keep_attrs=True)
temp = weighted_errors.mean(dim=['lat', 'lon'], skipna=True, keep_attrs=True)

# Revert the error variances back to error standard deviations
ds_et_results = xr.merge([ds_et_results, np.sqrt(temp)])

ds_et_results

### Comparison of TC Pre- and Post-Aggregation

Since the regional ET totals are technically collocated data sets, we can apply TC to estimate the error variances of these regional time series and compare to the error variances estimated by aggregating the gridded TC error variances from the [TC notebook](2_TC_application.ipynb#TC-Discussion) over the regions. This comparison can help give us an idea of how utilizing TC pre- and post-aggregating can affect the resulting error variance estimates.

First, we read in the EC function to compute the TC error estimates.

In [ ]:
%run ../TC/EC_function.ipynb

Then we generate the list all possible data set triplets.

In [ ]:
# Generate a list of the combinations
combos = list(itertools.combinations(dataset_name, 3))
combos = [list(combo) for combo in combos]

Since the data sets span different data ranges, we need to create a function that will limit the triplet to their common date range.

In [ ]:
def common_date_range(ds, combo):
    """Return the common date slice of the datasets."""
    old_common_date = []
    recent_common_date = []
    for name in combo:
        old_common_date.append(date_ranges[name][0])
        recent_common_date.append(date_ranges[name][1])
    
    return slice(np.max(old_common_date), np.min(recent_common_date))

Finally, we can compute the TC error and unbiased SNR estimates.

In [ ]:
# We want to ignore all of the sqrt and log warnings with negative values
warnings.filterwarnings("ignore", category=RuntimeWarning)

ec_var_est = []
for season in ds_results.season.data:
    if season == 'All':
        ds_season = ds_et_regional
    else:
        ds_season = ds_et_regional.isel(time=(ds_et_regional.time.dt.season == season))

    ec_var_est_combo = []
    for combo in combos:
        ds_combo = ds_season.sel(time=common_date_range(ds_et_regional, combo), dataset_name=combo)

        ec_covar, snr_temp = ec_covar_multi(ds_combo.aet.data, corr_sets=[1, 2, 3], return_snr=True)

        ec_var_est_combo.append(xr.Dataset(data_vars={'error': (['var_combo', 'season', 'est_idx', 'region_name'],
                                                                np.sqrt(np.diagonal(ec_covar).T)[None, None, ...]),
                                                      'snr': (['var_combo', 'season', 'est_idx', 'region_name'],
                                                              (10 ** np.log10(snr_temp[None, None, ...]))),},
                                           coords={'var_combo': [' '.join(combo)], 'season': [season], 
                                                   'est_idx': [0, 1, 2], 'region_name': ds_et_regional.region_name}))

    ec_var_est.append(xr.concat(ec_var_est_combo, dim='var_combo'))

ec_est = xr.concat(ec_var_est, dim='season')

# Restructure the data to be by data set vs by data set pair.
ec_est_by_dataset = []
est_pair = []
for name in dataset_name:
    idx_loc = np.char.find(ec_est.var_combo.data, name)
    dataset_loc = np.where(idx_loc != -1)[0]
    combos_single_dataset = ec_est.isel(var_combo=dataset_loc)
    ec_est_dataset = []
    for i in range(len(combos_single_dataset.var_combo.values)):
        ec_est_single_dataset = combos_single_dataset.isel(var_combo=i)
        idx = str(ec_est_single_dataset.var_combo.data).split(' ').index(name)
        ec_est_single_dataset = ec_est_single_dataset.sel(est_idx=idx)
        ec_est_single_dataset = ec_est_single_dataset.drop_vars(['var_combo', 'est_idx'])

        ec_est_dataset.append(xr.Dataset(data_vars=ec_est_single_dataset[['error', 'snr']],
                              coords={'dataset_name': name, 'est_idx': i, 'region_name': ec_est.region_name}))
    
    ec_est_dataset = xr.concat(ec_est_dataset, dim='est_idx')
    
    ec_est_by_dataset.append(ec_est_dataset)
    est_pair.append([combinations.replace(name, '').strip() for combinations in ec_est.var_combo.data[dataset_loc]])

ec_est = xr.concat(ec_est_by_dataset, dim='dataset_name')
ec_est = ec_est.assign_coords(est_pair=(['dataset_name', 'est_idx'], np.array(est_pair)))
ec_est

Now, let's make a plot that will compare the error estimates generated by TC pre- and post-aggregation to the regions. Actually, rather than comparing the error estimates, let's compare the unbiased SNRs. This will allow for an easier comparison, since the SNR is just unbaised signal divided by the error variance.

In [ ]:
tc_agg = xr.Dataset(data_vars={'pre-aggregation': ds_et_results.snr.rename({'var_est_idx': 'est_idx'}),
                               'post-aggregation': ec_est.snr})

def snr_plt(grouping='dataset_name'):

    coords = ['season', 'dataset_name', 'region_name']
    coords.remove(grouping)
    
    plt = (tc_agg.hvplot.bar(x=grouping, y=['pre-aggregation', 'post-aggregation'], groupby=['est_idx']+coords,
                            ylabel='Unbiased SNR', xlabel=' '.join(w.capitalize() for w in grouping.split('_')),
                            rot=90)
            * hv.HLine(1).opts(color='black', line_width=1, line_dash='dashed')).opts(frame_height=200)

    return pn.panel(plt, widget_location='top')

grouping_widget = pn.widgets.Select(name="grouping", value="dataset_name",
                                    options=['season', 'dataset_name', 'region_name'])

bound_plot = pn.bind(snr_plt, grouping=grouping_widget)

pn.Column(grouping_widget, bound_plot)

From this comparison, we can see that the SNR generated post-aggregation tends to have numerous `NaN` values, due to ET estimating negative error variances. This also occurred in the pre-aggregation maps, where serveral pixels in the region could have negative error variances. However, in our regional aggregation, we exclude these `NaN` values and only estimate the area weighted average using valid pixels. This process was cleanly hidden by `xarray.weighted.mean` as this method [automatically excludes `NaN` values and adjusts the weights accordingly](https://docs.xarray.dev/en/stable/user-guide/computation.html#comput-weighted). Therefore, let's continue to use the errors and SNRs that were generated from TC pre-aggregation as they have the `NaN` values averaged out. Also, using the pre-aggregated results retain the variation across the region rather then aggregating that variation out by aggregating before computing the TC errors.

## Discussion

### Regional TC Estimates

Since we decided to use the pre-aggregation TC error estimates, let's remake the above plot, but without the different calculation comparision. In other words, lets just plot the SNR of each data set for each season and region. This will allow us to explore what data sets are performing better (i.e., higher SNR) in each region and season. Also, since we are following the layout of the [TC notebook](2_TC_application.ipynb), let's plot the median of the different triplet estimates in addition to each individual estimate.

In [ ]:
ds_snr = ds_et_results.snr.to_dataset(dim='dataset_name').drop_vars('var_est_pair')

plt = (ds_snr.hvplot.bar(x='season', groupby=['region_name', 'var_est_idx'], ylabel='Unbiased SNR',
                            xlabel='Season', rot=90, ylim=(0, 25))
       * hv.HLine(1).opts(color='black', line_width=1, line_dash='dashed')
       * hv.HLine(5).opts(color='black', line_width=1, line_dash='dashed')).opts(frame_height=150)
plt =plt + (ds_snr
            .median(dim='var_est_idx')
            .hvplot.bar(x='season', groupby=['region_name'], ylabel='Median Unbiased SNR',
                        xlabel='Season', rot=90, ylim=(0, 25))
            * hv.HLine(1).opts(color='black', line_width=1, line_dash='dashed')
            * hv.HLine(5).opts(color='black', line_width=1, line_dash='dashed')).opts(frame_height=150)

pn.panel(plt.cols(1), widget_location='top')

From this comparison, we can see that in the Central Valley NLDAS, ERA5, and GLEAM have relatively high (>5) SNR for most seasons and combinations of data set triplets, whereas WBET has some high and low SNRs, and TerraClimate and SSEBop rarely have an SNR >1 at any point. This likely indicates that these three data sets (especially TerraClimate and SSEBop) are not optimized to function in this region, which could have implications on their use there. As for the High Plains Aquifer region, all data sets besides TerraClimate seem to have relativley high (>5) SNR values across all seaons. Looking at the seasons individually, most data sets have at least moderate (>1) SNR (besides TerraClimate). The SNR in winter is more variable and can have low (<1) SNR, which is expected. As we saw in the [TC notebook](2_TC_application.ipynb#TC-Estimation), the ET across CONUS is very low if not zero in for regions that actually experience a winter season (i.e. snow). Therefore, we would expect a low SNR for most if not all data sets during the winter months in both the High Plains and Upper Colorado. Finally, the Upper Colorado River Basin has a similar SNR pattern as the High Plains with the highest SNR in the Spring and Fall and lowest in the winter. For these two regions, no data sets really stand out as the optimal performer. Instead each seem to perform similarly well during each season.

### Regional EC Cross-Correlation Estimates

Using the pre-aggregation EC error estimates like the SNR plot, let's make another figure. This time of the EC error cross correlations of each data set for each season and region. This way we can check if any data sets are consistently correlated in our regions of interest. Again, to follow the previous [EC notebook](3_EC_application.ipynb), let's also plot the median cross correlation. We will not plot the cross-correlation SNR as we can easily flip through each estimate to get a view of how they vary.

In [ ]:
def agreement_by_dataset_plt(dataset_name='SSEBop', region_name='Central Valley', est_idx=0):

    idx_loc = np.char.find(ds_et_results.dataset_pair.data, dataset_name)
    ds_rho = (ds_et_results['rho']
              .isel(dataset_pair=(idx_loc != -1))
              .sel(region_name=region_name)
              .drop_vars(['est_pair']))
    ds_rho['dataset_pair'] = np.char.strip(
        np.char.replace(ds_rho['dataset_pair'].data, dataset_name, '')
    )
    ds_rho = ds_rho.to_dataset(dim='dataset_pair')

    plt = (ds_rho.sel(est_idx=est_idx).hvplot.bar(x='season', ylabel='Error Cross Correlation',
                                   xlabel='Season', rot=90, ylim=(-1, 1))
           * hv.HLine(0).opts(color='black', line_width=1, line_dash='dashed')).opts(frame_height=200, frame_width=500)

    plt = plt + (ds_rho.median(dim='est_idx').hvplot.bar(x='season', ylabel='Median Error Cross Correlation',
                                   xlabel='Season', rot=90, ylim=(-1, 1))
                 * hv.HLine(0).opts(color='black', line_width=1, line_dash='dashed')).opts(frame_height=200, frame_width=500)

    return plt.cols(2)

dataset_name_widget = pn.widgets.Select(name="dataset_name", value="SSEBop", options=dataset_name)
region_name_widget = pn.widgets.Select(name="region_name", value="Central Valley", options=list(ds_et_results.region_name.data))
est_idx_widget = pn.widgets.IntSlider(name='est_idx', start=0, end=5)

bound_plot = pn.bind(agreement_by_dataset_plt, dataset_name=dataset_name_widget, region_name=region_name_widget, est_idx=est_idx_widget)

pn.Column(dataset_name_widget, region_name_widget, est_idx_widget, bound_plot)

From the plots, we can see that the trends we saw in the maps in the [EC notebook](3_EC_application.ipynb) still hold. Those being SSEBop and WBET, along with ERA-5 and TerraClimate, have strong positive covariances, while SSEBop and TerraClimate have negative covariances. Besides these correlations, we can also see that GLEAM and ERA-5 seem to have positive correlations as well in all three regions. Therefore, this regional analysis allowed us to find another correlated pair.

### Data Set Agreement

Finally, let's look at the agreement between data sets for each region to see if those that have the higher SNRs (i.e., smaller errors) or strong correlations are in any less agreement with the other data sets. To do this, we will use the same method as in the [Bias notebook](4_Bias.ipynb#Relative-Bias-Discussion) to calculate the agreement probability. As a reminder, this method consists of:

1. Calculating the absolute relative bias ($\textrm{bias}_\textrm{abs} = | \textrm{dataset}_A - \textrm{dataset}_B |$),
2. [Propagating the data sets' error variances and covariance](https://en.wikipedia.org/wiki/Propagation_of_uncertainty#Example_formulae) to the bias uncertainty ($\sigma_{\varepsilon_{\rm bias}} = \sqrt{\sigma_{\varepsilon_A}^2 + \sigma_{\varepsilon_B}^2 - 2 \sigma_{\varepsilon_{AB}}}$), and
3. Calculating the probability density of a normal distribution of mean $\textrm{bias}_\textrm{abs}$ and variance $\sigma_{\varepsilon_{\rm bias}}^2$ that is less than or equal 0.

So, let's go ahead and calculate the agreement probabilities.

In [ ]:
variances = ds_et_results['covar'].linalg.diagonal(
                dims=['covar_pair_idx_1', 'covar_pair_idx_2'], offset=0,
            )
covariances = ds_et_results['covar'].linalg.diagonal(
                dims=['covar_pair_idx_1', 'covar_pair_idx_2'], offset=1,
              ).squeeze()
ds_et_results['sigma_bias'] = np.sqrt(variances.sum(dim='covar_pair_idx_1') - 2 * covariances)

norm_dist = stats.XrContinuousRV(norm,
                                 loc=np.abs(ds_et_results['median_bias']),
                                 scale=ds_et_results['sigma_bias'])
# Set and name it as a DataArray as the attributes of the coordinates are not kept
agreement_probability = norm_dist.cdf(0)
agreement_probability.name = 'agreement_probability'

# Merge to preserve coordinate attributes
ds_et_results = xr.merge([ds_et_results, agreement_probability])
ds_et_results

Now that we have the agreement probabilities, let's make a figure to visualize them.

In [ ]:
ds_agreement = ds_et_results['agreement_probability'].drop_vars('est_pair').to_dataset('dataset_pair')

plt = (ds_agreement.hvplot.bar(x='region_name', groupby=['season', 'est_idx'], ylabel='Agreement Probability',
                               xlabel='Region', rot=90, ylim=(0, 0.5))
       * hv.HLine(0.16).opts(color='black', line_width=1, line_dash='dashed')
       * hv.HLine(0.05).opts(color='black', line_width=1, line_dash='dashed')).opts(frame_height=200, frame_width=750)
pn.panel(plt, widget_location='top')

First from this figure, we can see that each of the agreement probability estimates for the data set pairs are very consistent, just as we saw in the [agreement notebook](4_dataset_agreement.ipynb). This is good as it shows that the variation in covariance matrices is not having a major influence on the agreement estimates. In terms of values, we can see that the High Plains and Upper Colorado have high agreement between all data sets when looking over all time (p > 0.16). When looking at their seasonal data, we can see that in winter and spring only about half of the data set pairs have high agreement (p > 0.16) and even some with low agreement (p < 0.05). This isn't unexpected as these regions can have snow during these months which can make ET estimates less reliable. Therefore, we would expect greater variation and bias during these months. As for the summer and fall, we see high agreement in almost all data sets in the High Plains and only a few low agreement probabilities in the Upper Colorado. Therefore, in these two regions, the data sets seem be in reasonable agreement within their errors.

However, this is not the case for the Central Valley. Overall time, the data sets do appear to be in high agreement (p > 0.16) except for NLDAS with ERA5, which have only moderate agreement (0.05 < p < 0.16). Yet, when looking at the seasons, any overall agreement seems to disappear. In the winter and summer, only a few data sets have any agreement, with the majority of data set pairs having extremely low agreement (<0.01). In the spring and fall, about two thirds of the data set pairs have high agreement, with the other third having moderate to low agreement (p > 0.16). This difference in agreement between seasonal and all season data is likely due to the way we aggregated our relative biases back in the [agreement notebook](4_dataset_agreement.ipynb). By aggregating all biases to the median across time, we are smoothing the variation across seasons. Therefore, it is likely that most ET data sets are not in agreement in the Central Valley in the peak ET season of summer, indicating the the choice of ET data set in the Central Valley can have impact on any results derived from it. 

Therefore, let's look deeper at the Central Valley to see how the agreement may be affecting the ET data sets in that region. Specifically, let's focus in on the 2012-2016 extreme drought period that devastated the region. First, let's plot the time series data again for that period, but with the associated median seasonal uncertainty included as a shaded area. Also, let's plot the cumulative ET of that time series to see how each data set compares in the estimated total ET during the drought.

In [ ]:
ds_drought = ds_et_regional.copy()

ds_drought['error'] = xr.zeros_like(ds_drought['aet'])
seasons = ds_drought.time.dt.season
for season in np.unique(seasons):
    ds_drought['error'] = ds_drought['error'].where(~(seasons == season),
                                                    ds_et_results.error.median(dim='var_est_idx').sel(season=season))

ds_drought['high'] = ds_drought['aet'] + ds_drought['error']
ds_drought['low'] = ds_drought['aet'] - ds_drought['error']
ds_drought = ds_drought.drop_vars('season')

ds_cumsum = ds_drought.aet.cumsum(dim='time').to_dataset()
ds_cumsum['aet'] = ds_cumsum.aet - ds_cumsum.aet.sel(time='10-01-2011')
ds_cumsum.aet.attrs['long_name'] = 'Cumulative AET'
ds_cumsum.aet.attrs['units'] = 'mm.month-1'

xlim = (pd.to_datetime('10-01-2011'), pd.to_datetime('09-30-2016'))
plt = ((ds_drought.aet.hvplot(groupby=['dataset_name', 'region_name'], xlim=xlim,
                              ylim=(-20, 150)).overlay('dataset_name').opts(legend_position='right')
        * ds_drought.hvplot.area(x='time', y='high', y2='low', groupby=['dataset_name', 'region_name'], alpha=0.5).overlay('dataset_name'))
       + (ds_cumsum.aet.hvplot(groupby=['dataset_name', 'region_name'], ylim=(-100, 2700),
                               xlim=xlim).overlay('dataset_name').opts(legend_position='right')))

pn.panel(plt.cols(1), widget_location='top')

Well, these are some very interesting results. As we saw above in the first time series plot, the Central Valley shows a misalignment between several of the data set. Even when including the errors, we do not see any better alignment or agreement between data sets. As stated above this offset is likely due to the high levels of irrigation in the Central Valley that are not being included in GLEAM, ERA5, NLDAS, and TerraClimate. Therefore, the misalignment in peak ET could explain some of the low agreement during the summer and potentially cause a serious bias issue on any derived product as the ET timings would be off by three months.

As for the cumulative ET plot, it is clear that there is a bias issue between each of the data sets, with WBET (SSEBop and ERA5) estimating almost double (one and a half) the ET as GLEAM, NLDAS, and TerraClimate. Since we have the errors on the time series, let's use that to generate some simulations to estimate if the differences in the cumulative distributions are statistically significant. We can do this with a simple two sample KS test.

In [ ]:
rng = np.random.default_rng()

ds_drought_ks = ds_drought.sel(time=slice('10-01-2011', '09-30-2016'))
x = rng.normal(loc=np.expand_dims(ds_drought_ks.aet, -1),
               scale=np.expand_dims(ds_drought_ks.error, -1),
               size=ds_drought_ks.error.shape + (1000,))
ds_drought_ks['aet_sim'] = (['time', 'dataset_name', 'region_name', 'sim'], x)

pairs = list(itertools.combinations(dataset_name, 2))
pairs = [list(pair) for pair in pairs]

for region_name in ds_drought_ks.region_name.data:
    ds_region = ds_drought_ks.sel(region_name=region_name)
    print(region_name)
    for pair in pairs:
        results = kstest(ds_region.aet_sim.sel(dataset_name=pair[0]), ds_region.aet_sim.sel(dataset_name=pair[1]))
        print(f'K-S Test results for {pair[0]} and {pair[1]} p-value = {np.median(results[1]):.4f} +/- {np.std(results[1]):.4f}')
    print()

From the KS tests, we can see that as per the plot only GLEAM, NLDAS, and TerraClimate cannot be deemed to have a statistically significance difference (p < 0.05) in their cumulative ET. Even SSEBop and ERA5 have different cumulative ET, while visually looking relatively similar. This disagreement is likely from the offset in the time series causing the significant difference. Therefore, the differences in the bias of these ET data sets should be taken into consideration when utilizing them in other studies.

As a final thing to check, if we zoom out time-wise on the cumulative distributions, we can see that each data set seems to follow a constant slope. If this is the case, we could easily bias correct these data sets to some chosen standard. While exploring this further is beyond these notebooks, let's just make one last plot to see if it works.

In [ ]:
ds_cumsum_fit = ds_drought.sel(time=slice('10-01-2011', '09-30-2016')).aet.cumsum(dim='time')
ds_cumsum_fit['time'] = list(range(len(ds_cumsum_fit['time'])))
fit_results = ds_cumsum_fit.curvefit('time', lambda params, x: params * x).squeeze()

# For this test, we will choose WBET as our standard to correct to
ds_drought_bias_adj = (fit_results.curvefit_coefficients.sel(dataset_name='WBET')
                       / fit_results.curvefit_coefficients
                       * ds_drought.aet)
ds_drought_bias_adj.attrs['long_name'] = 'Bias corrected AET'
ds_drought_bias_adj.attrs['units'] = 'mm.month-1'

plt = (ds_drought.aet.hvplot(
           groupby=['dataset_name', 'region_name'], ylim=(-20, 160), xlim=xlim
       ).overlay('dataset_name').opts(legend_position='right')
       + ds_drought_bias_adj.hvplot(
           groupby=['dataset_name', 'region_name'], ylim=(-20, 160), xlim=xlim
       ).overlay('dataset_name').opts(legend_position='right')
       + (ds_drought_bias_adj.cumsum(dim='time') - ds_drought_bias_adj.cumsum(dim='time').sel(time='10-01-2011')).hvplot(
           groupby=['dataset_name', 'region_name'], ylim=(-100, 2600), xlim=xlim
       ).overlay('dataset_name').opts(legend_position='right'))

pn.panel(plt.cols(1), widget_location='top')

Hey look at that, it seems like this kind of bias correction may work! So, these data sets could be corrected. One last thing to note is that while the raw data sets may likely need a bias correction, the TC error estimates that were at the center of this work should not change after a bias correction. As you can check and remember from the [TC function](../TC/TC_function.ipynb), the TC method we used assumes an affine error model that will include any linear biases that maybe present between data sets. Therefore, even in the presence of this bias, the error estimates made from the TC method should remain unchanged.